In [ ]:
#a stable XGBoost version that works well with sklearn Pipeline
!pip -q install xgboost==1.7.6

In [ ]:
# Cell 0 — Setup (Colab-friendly)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass  # not Colab

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/renewables_outputs/merged_by_continent/refined_df.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 1 — Load CSV, map columns, quick hygiene + indicators as categorical-ready ints
import pandas as pd
import numpy as np

df = pd.read_csv(DATA_PATH)
print("Loaded:", df.shape)

COLS = {
    "timestamp": ["timestamp","ts_utc","UTC","datetime","time","date_time","ts"],
    "site":      ["site","site_id","region","zone","location"],
    "solar":     ["target_solar_mw","solar_generation","solar_mw","pv_mw"],
    "wind":      ["target_wind_mw","wind_generation","wind_mw"],
}

def first_existing(cols, candidates):
    for c in candidates:
        if c in cols: return c
    return None

ts_col   = first_existing(df.columns, COLS["timestamp"]); assert ts_col, "timestamp column not found"
site_col = first_existing(df.columns, COLS["site"]);      assert site_col, "site column not found"
solar_col = first_existing(df.columns, COLS["solar"])
wind_col  = first_existing(df.columns, COLS["wind"])

df[ts_col] = pd.to_datetime(df[ts_col], utc=True, errors="coerce")
df = df.dropna(subset=[ts_col, site_col]).sort_values([site_col, ts_col]).reset_index(drop=True)

# Indicators → 0/1 ints (we will treat them as categorical via OHE later)
INDICATORS = ["is_weekend", "is_daylight", "solar_struct_zero", "wind_near_zero"]
for c in INDICATORS:
    if c in df.columns:
        if df[c].dtype == bool:
            df[c] = df[c].astype("uint8")
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).clip(0, 1).astype("uint8")

# --- De-duplicate any repeated column names (keep first) ---
dups = df.columns[df.columns.duplicated()].tolist()
if dups:
    print("Deduplicated repeated columns:", sorted(set(dups)))
    df = df.loc[:, ~df.columns.duplicated()].copy()

print({"rows": len(df), "sites": df[site_col].nunique(), "targets": {"solar": solar_col, "wind": wind_col}})

Loaded: (2698080, 24)
{'rows': 2698080, 'sites': 154, 'targets': {'solar': 'target_solar_mw', 'wind': 'target_wind_mw'}}


In [ ]:
# Cell 2 — Speed knobs & shared helpers
import os, importlib
import numpy as np

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pyrandom = importlib.import_module('random')
pyrandom.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)

QUICK          = True           # switch to False for final runs
HORIZONS       = [1, 24] if QUICK else [1, 3, 6, 24]
N_SPLITS       = 3 if QUICK else 5
N_ESTIMATORS   = 160 if QUICK else 400
MAX_DEPTH      = 5 if QUICK else 7
LEARNING_RATE  = 0.15 if QUICK else 0.08
MAX_NUM_COLS   = 40            # cap numeric features by variance for speed
SITES_FILTER   = None          # e.g., {"DE_Berlin","US_Denver"} for quick iteration
RANDOM_STATE   = 42

if SITES_FILTER:
    df = df[df[site_col].isin(SITES_FILTER)].copy()

from sklearn.metrics import mean_absolute_error, mean_squared_error

def eval_metrics(y_true, y_pred):
    y = np.asarray(y_true, dtype=float); yhat = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y) & np.isfinite(yhat)
    if not np.any(mask):
        return {"MAE": np.nan, "RMSE": np.nan, "MAPE%": np.nan, "sMAPE%": np.nan, "nMAE": np.nan}
    y = y[mask]; yhat = yhat[mask]
    mae  = mean_absolute_error(y, yhat)
    rmse = float(np.sqrt(mean_squared_error(y, yhat)))  # manual sqrt (no squared=False)
    mape = float(np.mean(np.abs((y - yhat) / np.clip(np.abs(y), 1e-6, None))) * 100.0)
    smape= float(np.mean(2.0 * np.abs(y - yhat) / (np.abs(y) + np.abs(yhat) + 1e-6)) * 100.0)
    nmae = float(mae / (np.mean(np.abs(y)) + 1e-6))
    return {"MAE": mae, "RMSE": rmse, "MAPE%": mape, "sMAPE%": smape, "nMAE": nmae}

# Zero-aware % metrics: only score % errors where it makes sense (avoid zeros)
def eval_metrics_zeroaware(y_true, y_pred, task: str, te_df):
    y = np.asarray(y_true, dtype=float)
    yhat = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y) & np.isfinite(yhat)

    if task.lower() == "solar":
        if "is_daylight" in te_df.columns:
            mask &= te_df["is_daylight"].to_numpy().astype(bool)
        if "solar_struct_zero" in te_df.columns:
            mask &= ~te_df["solar_struct_zero"].to_numpy().astype(bool)
        # fallback if no flags at all: only evaluate where |y|>1e-6
        if ("is_daylight" not in te_df.columns) and ("solar_struct_zero" not in te_df.columns):
            mask &= (np.abs(y) > 1e-6)

    elif task.lower() == "wind":
        if "wind_near_zero" in te_df.columns:
            mask &= ~te_df["wind_near_zero"].to_numpy().astype(bool)
        else:
            mask &= (np.abs(y) > 1e-6)

    else:
        # default fallback
        mask &= (np.abs(y) > 1e-6)

    if not np.any(mask):
        return {"MAPE%_ZA": np.nan, "sMAPE%_ZA": np.nan}

    y = y[mask]; yhat = yhat[mask]
    mape = float(np.mean(np.abs((y - yhat) / np.clip(np.abs(y), 1e-6, None))) * 100.0)
    smape = float(np.mean(2.0 * np.abs(y - yhat) / (np.abs(y) + np.abs(yhat) + 1e-6)) * 100.0)
    return {"MAPE%_ZA": mape, "sMAPE%_ZA": smape}

def seasonal_naive_daily(series, H):  return series.shift(24 - H)
def seasonal_naive_weekly(series, H): return series.shift(168 - H)

def choose_target(task):
    if task == "solar" and solar_col: return solar_col
    if task == "wind"  and wind_col:  return wind_col
    raise ValueError(f"Target for task={task} not found.")

N_SPLITS = 6

In [ ]:
# Cell 3 — Make supervised (robust to duplicate target cols) + blocked splits
def _resolve_target_series(frame: pd.DataFrame, target_name: str) -> pd.Series:
    """Return a single numeric Series for the target, even if duplicate-named columns exist."""
    t = frame.loc[:, target_name]
    if isinstance(t, pd.DataFrame):
        # pick the duplicate column with most non-nulls; tie-breaker: highest variance
        non_null = t.notna().sum()
        cand = non_null.idxmax()
        t = t[cand]
    return pd.to_numeric(t, errors="coerce")

def make_supervised(df_in: pd.DataFrame, target: str, H: int):
    """
    gH: label y = future H (per site).
    Numeric pool is variance-capped and EXCLUDES indicators so they go through the categorical pipeline.
    """
    g = df_in.copy()

    # robust target series + horizon shift by site
    t_series = _resolve_target_series(g, target)
    g["y"] = t_series.groupby(g[site_col]).shift(-H)

    # numeric candidates (drop targets, label, and indicators)
    num_all = g.select_dtypes(include=[np.number]).columns.tolist()
    drop_like = {"y"}
    for c in [solar_col, wind_col]:
        if c: drop_like.add(c)
    drop_like.update([c for c in ["is_weekend","is_daylight","solar_struct_zero","wind_near_zero"] if c in g.columns])
    num_cols = [c for c in num_all if c not in drop_like]
    const_mask = (g[num_cols].nunique(dropna=False) <= 1)
    if const_mask.any():
       num_cols = [c for c in num_cols if not const_mask.get(c, False)]

    # cap by variance for speed
    if len(num_cols) > MAX_NUM_COLS:
        var_rank = g[num_cols].var().sort_values(ascending=False)
        num_cols = list(var_rank.index[:MAX_NUM_COLS])

    # categorical = site + indicators (if present)
    cat_cols = [c for c in [site_col, "is_weekend","is_daylight","solar_struct_zero","wind_near_zero"] if c in g.columns]

    Xc = num_cols + cat_cols
    yc = "y"
    gH = g.dropna(subset=[yc]).copy()
    return gH, Xc, yc, num_cols, cat_cols

def blocked_splits_by_time(df_in: pd.DataFrame, n_splits: int, gap_hours: int = 24):
    times = np.array(sorted(df_in[ts_col].unique()))
    blocks = np.array_split(times, n_splits)
    for i in range(1, n_splits):
        tr_times = np.concatenate(blocks[:i])
        te_times = blocks[i]
        # gap: drop final 'gap_hours' worth of timestamps from train
        if len(tr_times) and len(te_times):
            cutoff = te_times.min() - pd.Timedelta(hours=gap_hours)
            tr_times = tr_times[tr_times <= cutoff]
        yield tr_times, te_times

# robustly resolve a single target Series even if the name is duplicated
def _resolve_target_series(frame: pd.DataFrame, target_name: str) -> pd.Series:
    """
    Return a single numeric Series for the target, even if there are duplicate-named columns.
    Strategy: find all column indices matching `target_name`, choose the one with the
    most non-nulls; tie-breaker = highest variance (over non-nulls).
    """
    # all positions with this column name
    col_idxs = [i for i, c in enumerate(frame.columns) if c == target_name]
    if not col_idxs:
        raise KeyError(f"Target '{target_name}' not found in frame.columns")

    if len(col_idxs) == 1:
        s = frame.iloc[:, col_idxs[0]]
    else:
        sub = frame.iloc[:, col_idxs]              # DataFrame of duplicates
        nn = sub.notna().sum(axis=0).to_numpy()    # non-null counts per dup col
        best = int(np.nanargmax(nn))
        # tie-break among equal nn by variance
        ties = np.where(nn == nn[best])[0]
        if len(ties) > 1:
            variances = sub.iloc[:, ties].var(axis=0, skipna=True).to_numpy()
            best = int(ties[int(np.nanargmax(variances))])
        s = sub.iloc[:, best]

    # guarantee 1-D numeric series
    return pd.to_numeric(s, errors="coerce")


In [ ]:
# run_blocked_cv (Cell 4): fast, single preprocessor per fold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy import sparse
from xgboost import XGBRegressor

# Initialize cv_site_table_all before the function is called
cv_site_table_all = pd.DataFrame()


def build_preprocessor_sparse(num_cols, cat_cols):
    try:
        ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
    num_trans = Pipeline([("imp", SimpleImputer(strategy="median")),
                          ("sc",  StandardScaler(with_mean=False))])
    cat_trans = Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", ohe)])
    return ColumnTransformer(
        [("num", num_trans, num_cols),
         ("cat", cat_trans, cat_cols)],
        remainder="drop", sparse_threshold=1.0
    )

def _split_train_val_idx(n, val_frac=0.15):
    v = max(50, int(round(n * (1 - val_frac))))
    return np.arange(0, v), np.arange(v, n)

def run_blocked_cv(df_in: pd.DataFrame, task: str, horizons=HORIZONS, n_splits=N_SPLITS, gap_hours: int = 24):
    target = choose_target(task)
    rows, rows_site = [], []

    base = df_in[[ts_col, site_col, target] + df_in.select_dtypes(include=[np.number]).columns.tolist()].copy()
    base = base.loc[:, ~base.columns.duplicated()].sort_values(ts_col)

    for H in horizons:
        gH, Xc, yc, num_cols, cat_cols = make_supervised(base, target, H)
        if gH.empty:
            continue

        # helper for aligned baselines
        def _target_series(frame): return _resolve_target_series(frame, target)

        for fold, (tr_times, te_times) in enumerate(blocked_splits_by_time(gH, n_splits, gap_hours), 1):
            tr = gH[gH[ts_col].isin(tr_times)].copy()
            te = gH[gH[ts_col].isin(te_times)].copy()
            if len(tr) < 200 or len(te) < 100:
                continue

            # ---- 1) Fit ONE preprocessor, transform once
            pre = build_preprocessor_sparse(num_cols, cat_cols)
            Xt_tr = pre.fit_transform(tr[Xc])
            Xt_te = pre.transform(te[Xc])
            y_tr  = tr[yc].to_numpy()
            y_te  = te[yc].to_numpy()

            # ---- 2) Split train->(train/val) for early stopping (fallback-safe)
            tr_idx, va_idx = _split_train_val_idx(Xt_tr.shape[0], val_frac=0.15)
            Xt_tr_, y_tr_  = Xt_tr[tr_idx], y_tr[tr_idx]
            Xt_va_, y_va_  = Xt_tr[va_idx], y_tr[va_idx]

            # ---- 3) Optional: per-site **sample weights** (pooled fairness)
            # inverse frequency so large sites don't dominate
            counts = tr[site_col].value_counts()
            w = tr[site_col].map(lambda s: 1.0 / max(1, counts.get(s, 1))).astype(float).to_numpy()
            w = w / w.mean()  # normalize
            w_tr_, w_va_ = w[tr_idx], w[va_idx]

            # ---- 4) Small speed/quality grid (pick best on val RMSE; version-safe)
            param_grid = [
                {"n_estimators": 160, "max_depth": 5, "learning_rate": 0.15},  # fast & small
                {"n_estimators": 300, "max_depth": 6, "learning_rate": 0.10},  # a bit larger
                #{"n_estimators": 220, "max_depth": 5, "learning_rate": 0.12},
              ]

            best_rmse = float("inf")
            best_model = None

            for p in param_grid:
                mdl = XGBRegressor(
                    subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                    tree_method="hist", n_jobs=-1, random_state=RANDOM_STATE, verbosity=0,
                    **p
                )
                # Plain fit (no early stopping to avoid version issues)
                mdl.fit(Xt_tr_, y_tr_, sample_weight=w_tr_)
                # Manual validation RMSE
                pred_va = mdl.predict(Xt_va_)
                rmse_va = float(np.sqrt(((pred_va - y_va_) ** 2).mean()))
                if rmse_va < best_rmse:
                    best_rmse = rmse_va
                    best_model = mdl

            # Final test prediction with the best model from the tiny grid
            pred = best_model.predict(Xt_te)

            # baselines (aligned to y_{t+H})
            t_te    = _target_series(te)
            pers    = t_te.groupby(te[site_col]).shift(0).to_numpy()
            seas_dy = t_te.groupby(te[site_col]).shift(24 - H).to_numpy()
            seas_wk = t_te.groupby(te[site_col]).shift(168 - H).to_numpy()
            za_xgb  = eval_metrics_zeroaware(y_te, pred,    task, te)
            za_pers = eval_metrics_zeroaware(y_te, pers,    task, te)
            za_dy   = eval_metrics_zeroaware(y_te, seas_dy, task, te)
            za_wk   = eval_metrics_zeroaware(y_te, seas_wk, task, te)

            # ---- fold-level metrics
            rows.append({
                "task": task, "target_col": target, "fold": fold, "horizon": H,
                **{f"XGB_{k}": v for k, v in eval_metrics(y_te, pred).items()},
                **{f"Pers_{k}": v for k, v in eval_metrics(y_te, pers).items()},
                **{f"SeasDay_{k}": v for k, v in eval_metrics(y_te, seas_dy).items()},
                **{f"SeasWk_{k}": v for k, v in eval_metrics(y_te, seas_wk).items()},
                "XGB_MAPE%_ZA":    za_xgb["MAPE%_ZA"],   "XGB_sMAPE%_ZA":    za_xgb["sMAPE%_ZA"],
                "Pers_MAPE%_ZA":   za_pers["MAPE%_ZA"],  "Pers_sMAPE%_ZA":   za_pers["sMAPE%_ZA"],
                "SeasDay_MAPE%_ZA":za_dy["MAPE%_ZA"],    "SeasDay_sMAPE%_ZA":za_dy["sMAPE%_ZA"],
                "SeasWk_MAPE%_ZA": za_wk["MAPE%_ZA"],    "SeasWk_sMAPE%_ZA": za_wk["sMAPE%_ZA"],
                "n_train": len(tr), "n_test": len(te),
                "features_num": len(num_cols), "features_cat": len(cat_cols)
            })

            # ---- site×fold metrics (keeps your stronger stats/power)
            df_te = pd.DataFrame({
                "site": te[site_col].to_numpy(),
                "y":    y_te,
                "xgb":  pred,
                "pers": pers,
                "dy":   seas_dy,
                "wk":   seas_wk,
            })
            for sid, g in df_te.groupby("site"):
                m_xgb  = eval_metrics(g["y"], g["xgb"])
                m_pers = eval_metrics(g["y"], g["pers"])
                m_dy   = eval_metrics(g["y"], g["dy"])
                m_wk   = eval_metrics(g["y"], g["wk"])
                rows_site.append({
                    "task": task, "target_col": target, "fold": fold, "horizon": H, "site": sid,
                    "XGB_RMSE":  m_xgb["RMSE"],  "XGB_MAE":  m_xgb["MAE"],
                    "Pers_RMSE": m_pers["RMSE"], "Pers_MAE": m_pers["MAE"],
                    "SeasDay_RMSE": m_dy["RMSE"], "SeasDay_MAE": m_dy["MAE"],
                    "SeasWk_RMSE":  m_wk["RMSE"], "SeasWk_MAE":  m_wk["MAE"],
                    "n_site_test": len(g)
                })

    # expose site-wise table for both tasks (append, not overwrite)
    global cv_site_table_all, cv_site_table_last
    cv_site_table_all = (pd.concat([cv_site_table_all, pd.DataFrame(rows_site)], ignore_index=True)
                         if 'cv_site_table_all' in globals() and not pd.DataFrame(rows_site).empty
                         else pd.DataFrame(rows_site))
    cv_site_table_last = pd.DataFrame(rows_site)


    return pd.DataFrame(rows)

In [ ]:
# Cell 5 — Run blocked CV for available targets
tables = []
if solar_col: tables.append(run_blocked_cv(df, "solar"))
if wind_col:  tables.append(run_blocked_cv(df, "wind"))
cv_table = pd.concat(tables, ignore_index=True) if tables else pd.DataFrame()

print(f"Done. CV rows: {len(cv_table)} | QUICK={QUICK} | HORIZONS={HORIZONS} | SPLITS={N_SPLITS} | num_cap={MAX_NUM_COLS}")
display(cv_table.head(12))

# Saving report
out_path = "/content/drive/MyDrive/Colab Notebooks/out/cv_results_with_indicators_cat.csv"
cv_table.to_csv(out_path, index=False)
print("Saved:", out_path)


Done. CV rows: 20 | QUICK=True | HORIZONS=[1, 24] | SPLITS=6 | num_cap=40


,task,target_col,fold,horizon,XGB_MAE,XGB_RMSE,XGB_MAPE%,XGB_sMAPE%,XGB_nMAE,Pers_MAE,...,Pers_MAPE%_ZA,Pers_sMAPE%_ZA,SeasDay_MAPE%_ZA,SeasDay_sMAPE%_ZA,SeasWk_MAPE%_ZA,SeasWk_sMAPE%_ZA,n_train,n_test,features_num,features_cat
0,solar,target_solar_mw,1,1,0.021108,0.038900,1.253334e+05,95.037049,0.109964,0.052183,...,1.088697e+05,55.725331,1.425470e+02,24.863527,5.613926e+02,30.289171,446138,449680,14,5
1,solar,target_solar_mw,2,1,0.017167,0.032730,1.344385e+05,118.467858,0.142920,0.038323,...,1.518004e+05,62.977555,2.641227e+02,29.000369,1.717184e+03,37.643131,895818,449680,14,5
2,solar,target_solar_mw,3,1,0.013588,0.027807,1.060557e+05,110.973228,0.095641,0.043203,...,1.632860e+05,61.034728,1.241126e+02,28.894962,1.636435e+02,37.135341,1345498,449680,14,5
3,solar,target_solar_mw,4,1,0.016698,0.033727,7.682464e+04,90.636112,0.086590,0.051967,...,1.105238e+05,55.434328,1.306186e+02,24.180507,5.557672e+02,29.711282,1795178,449680,14,5
4,solar,target_solar_mw,5,1,0.011759,0.024845,1.039261e+05,115.720305,0.097009,0.038141,...,1.530009e+05,62.592715,2.870853e+02,28.494439,1.716555e+03,36.842636,2244858,449526,14,5
5,solar,target_solar_mw,1,24,0.047505,0.080404,4.725680e+05,101.446842,0.247421,0.037563,...,4.501083e+01,25.904416,4.501083e+01,25.904416,5.416981e+01,30.889342,445522,449064,14,5
6,solar,target_solar_mw,2,24,0.027071,0.056334,1.436192e+05,121.110284,0.226152,0.025924,...,8.079505e+01,30.647670,8.079505e+01,30.647670,1.255655e+02,38.353254,894586,449064,14,5
7,solar,target_solar_mw,3,24,0.028995,0.062002,5.770993e+04,114.024156,0.203751,0.030646,...,4.425840e+01,30.454242,4.425840e+01,30.454242,4.890553e+01,38.833810,1343650,449064,14,5
8,solar,target_solar_mw,4,24,0.033865,0.067093,4.580985e+04,93.475650,0.175560,0.036502,...,3.880505e+01,25.229245,3.880505e+01,25.229245,4.327694e+01,30.351671,1792714,449064,14,5
9,solar,target_solar_mw,5,24,0.023219,0.051184,3.984140e+04,117.713212,0.191683,0.024606,...,6.390865e+01,30.000767,6.390865e+01,30.000767,8.968041e+01,37.494743,2241778,449064,14,5


Saved: /content/drive/MyDrive/Colab Notebooks/out/cv_results_with_indicators_cat.csv


In [ ]:
# Cell 5a — Summaries & deltas vs best baseline (per task/horizon)
import pandas as pd
import numpy as np

def _best_baseline(df, metric="RMSE"):
    cols = [f"Pers_{metric}", f"SeasDay_{metric}", f"SeasWk_{metric}"]
    best_val = df[cols].min(axis=1)
    best_name = df[cols].idxmin(axis=1).str.replace(f"_{metric}","", regex=False)
    return best_name, best_val

# Compute best baseline RMSE and delta
best_name, best_rmse = _best_baseline(cv_table, "RMSE")
cv_table = cv_table.copy()
cv_table["best_base_name"] = best_name
cv_table["best_base_RMSE"] = best_rmse
cv_table["delta_RMSE"] = cv_table["best_base_RMSE"] - cv_table["XGB_RMSE"]  # + = model better
cv_table["win"] = (cv_table["delta_RMSE"] > 0).astype(int)

summary = (cv_table
           .groupby(["task","horizon"], as_index=False)
           .agg(pairs=("fold","count"),
                win_rate=("win", "mean"),
                mean_XGB_RMSE=("XGB_RMSE","mean"),
                mean_best_base_RMSE=("best_base_RMSE","mean"),
                mean_delta_RMSE=("delta_RMSE","mean"),
                mean_XGB_MAE=("XGB_MAE","mean"))
          )

print("Summary vs best baseline (fold-level):")
display(summary)

#drop noisy MAPE columns when browsing the big table
cols_to_drop = [c for c in cv_table.columns if "_MAPE%" in c]
tidy_cv = cv_table.drop(columns=cols_to_drop)
print("Tidy CV table (no MAPE columns):")
display(tidy_cv.head(12))


Summary vs best baseline (fold-level):


,task,horizon,pairs,win_rate,mean_XGB_RMSE,mean_best_base_RMSE,mean_delta_RMSE,mean_XGB_MAE
0,solar,1,5,1.0,0.031602,0.071250,0.039649,0.016064
1,solar,24,5,1.0,0.063403,0.073698,0.010294,0.032131
2,wind,1,5,1.0,0.580910,0.674414,0.093504,0.391741
3,wind,24,5,1.0,0.895057,1.175206,0.280149,0.831194


Tidy CV table (no MAPE columns):


,task,target_col,fold,horizon,XGB_MAE,XGB_RMSE,XGB_sMAPE%,XGB_nMAE,Pers_MAE,Pers_RMSE,...,SeasDay_sMAPE%_ZA,SeasWk_sMAPE%_ZA,n_train,n_test,features_num,features_cat,best_base_name,best_base_RMSE,delta_RMSE,win
0,solar,target_solar_mw,1,1,0.021108,0.038900,95.037049,0.109964,0.052183,0.078795,...,24.863527,30.289171,446138,449680,14,5,Pers,0.078795,0.039896,1
1,solar,target_solar_mw,2,1,0.017167,0.032730,118.467858,0.142920,0.038323,0.066165,...,29.000369,37.643131,895818,449680,14,5,SeasDay,0.064858,0.032128,1
2,solar,target_solar_mw,3,1,0.013588,0.027807,110.973228,0.095641,0.043203,0.072101,...,28.894962,37.135341,1345498,449680,14,5,Pers,0.072101,0.044294,1
3,solar,target_solar_mw,4,1,0.016698,0.033727,90.636112,0.086590,0.051967,0.078254,...,24.180507,29.711282,1795178,449680,14,5,Pers,0.078254,0.044528,1
4,solar,target_solar_mw,5,1,0.011759,0.024845,115.720305,0.097009,0.038141,0.065693,...,28.494439,36.842636,2244858,449526,14,5,SeasDay,0.062243,0.037398,1
5,solar,target_solar_mw,1,24,0.047505,0.080404,101.446842,0.247421,0.037563,0.083674,...,25.904416,30.889342,445522,449064,14,5,Pers,0.083674,0.003270,1
6,solar,target_solar_mw,2,24,0.027071,0.056334,121.110284,0.226152,0.025924,0.064906,...,30.647670,38.353254,894586,449064,14,5,Pers,0.064906,0.008572,1
7,solar,target_solar_mw,3,24,0.028995,0.062002,114.024156,0.203751,0.030646,0.075104,...,30.454242,38.833810,1343650,449064,14,5,Pers,0.075104,0.013102,1
8,solar,target_solar_mw,4,24,0.033865,0.067093,93.475650,0.175560,0.036502,0.082512,...,25.229245,30.351671,1792714,449064,14,5,Pers,0.082512,0.015419,1
9,solar,target_solar_mw,5,24,0.023219,0.051184,117.713212,0.191683,0.024606,0.062292,...,30.000767,37.494743,2241778,449064,14,5,Pers,0.062292,0.011108,1


In [ ]:
# Cell 6 — RQ1 (site×fold pairs): improvements vs best baseline + Wilcoxon + sign test
from scipy.stats import wilcoxon, binomtest

def _best_base_vals(row, metric="RMSE"):
    vals = {
        "Pers":   row.get(f"Pers_{metric}"),
        "SeasDay":row.get(f"SeasDay_{metric}"),
        "SeasWk": row.get(f"SeasWk_{metric}"),
    }
    vals = {k:v for k,v in vals.items() if pd.notnull(v)}
    if not vals: return None, np.nan
    best = min(vals, key=vals.get)
    return best, vals[best]

def rq1_from_site_pairs(tbl):
    rows = []
    for (task, H), grp in tbl.groupby(["task","horizon"]):
        deltas = []
        wins = 0
        for _, r in grp.iterrows():
            best_name, best_rmse = _best_base_vals(r, "RMSE")
            if best_name is None:
                continue
            deltas.append(best_rmse - r["XGB_RMSE"])
            wins += int(r["XGB_RMSE"] < best_rmse)
        n = len(deltas)
        if n == 0:
            continue
        # Wilcoxon (paired, H1: model < baseline)
        try:
            stat_w, p_w = wilcoxon(deltas, alternative="greater")
        except ValueError:
            p_w = np.nan
        # Sign test (exact binomial, robust for small n)
        p_sign = binomtest(wins, n, 0.5, alternative="greater").pvalue
        rows.append({
            "task": task, "horizon": H,
            "pairs(site×fold)": n,
            "wins": wins, "win_rate": wins / n,
            "mean_delta_RMSE": float(np.mean(deltas)),
            "wilcoxon_p": p_w,
            "sign_p": p_sign
        })
    return pd.DataFrame(rows).sort_values(["task","horizon"])

# Prefer sitewise table if available; else fall back to fold-level table
if 'cv_site_table_last' in globals() and not cv_site_table_last.empty:
    rq1_summary = rq1_from_site_pairs(cv_site_table_last)
else:
    # Fallback: your original fold-level summary (kept for completeness)
    rq1_summary = pd.DataFrame([{
        "task":"-", "horizon":"-", "pairs(site×fold)":0, "wins":0,
        "win_rate":0.0, "mean_delta_RMSE":np.nan, "wilcoxon_p":np.nan, "sign_p":np.nan
    }])

display(rq1_summary)


,task,horizon,pairs(site×fold),wins,win_rate,mean_delta_RMSE,wilcoxon_p,sign_p
0,wind,1,770,768,0.997403,0.091965,5.522189e-128,4.779912e-227
1,wind,24,770,767,0.996104,0.275546,5.652936e-128,1.225259e-224


In [ ]:
# Cell 6b — RQ1 significance (Wilcoxon + Sign test + BH-FDR)

import numpy as np, pandas as pd
from scipy.stats import wilcoxon, binomtest

def bh_fdr(pvals, alpha=0.05):
    """Benjamini–Hochberg FDR adjust; returns adj p-values in same order."""
    p = np.asarray(pvals, dtype=float)
    n = len(p); order = np.argsort(p); ranked = p[order]
    adj = np.empty_like(ranked)
    cum = 0.0
    for i in range(n-1, -1, -1):
        cum = min(cum if cum else 1.0, ranked[i] * n / (i+1))
        adj[i] = cum
    out = np.empty_like(adj); out[order] = adj
    return out

def paired_summary(deltas, wins):
    n = len(deltas)
    if n == 0:
        return dict(p_wilcoxon=np.nan, p_sign=np.nan, effect_d=np.nan, n=n, wins=wins)
    # Wilcoxon (H1: model < best-baseline => deltas > 0)
    try:
        _, p_w = wilcoxon(deltas, alternative="greater")
    except ValueError:
        p_w = np.nan
    # Exact sign test
    p_s = binomtest(wins, n, 0.5, alternative="greater").pvalue
    # Paired Cohen's d (mean/sd of deltas)
    sd = np.std(deltas, ddof=1) if n > 1 else np.nan
    d  = float(np.mean(deltas) / sd) if (sd and sd > 0) else np.nan
    return dict(p_wilcoxon=p_w, p_sign=p_s, effect_d=d, n=n, wins=wins)

def best_baseline_vals_row(row, metric="RMSE"):
    vals = [row.get(f"Pers_{metric}"), row.get(f"SeasDay_{metric}"), row.get(f"SeasWk_{metric}")]
    return np.nanmin(vals)

# -------- Fold-level significance (uses cv_table) --------
rows = []
for (task, H), grp in cv_table.groupby(["task","horizon"]):
    deltas = (grp["best_base_RMSE"] - grp["XGB_RMSE"]).to_numpy()  # + = model better
    wins   = int(np.sum(deltas > 0))
    summ   = paired_summary(deltas, wins)
    rows.append({"level":"fold", "task":task, "horizon":int(H), "pairs":summ["n"],
                 "wins":summ["wins"], "win_rate": wins/max(1,summ["n"]),
                 "mean_delta_RMSE": float(np.mean(deltas)),
                 "p_wilcoxon": summ["p_wilcoxon"], "p_sign": summ["p_sign"], "effect_d": summ["effect_d"]})
rq1_sig_fold = pd.DataFrame(rows).sort_values(["task","horizon"]).reset_index(drop=True)

# -------- Site×fold significance (uses cv_site_table_last if present) --------
if "cv_site_table_last" in globals() and not cv_site_table_last.empty:
    rows = []
    gtbl = cv_site_table_last.copy()
    # compute best-baseline RMSE per row
    base_rmse = np.nanmin(np.vstack([
        gtbl["Pers_RMSE"].to_numpy(),
        gtbl["SeasDay_RMSE"].to_numpy(),
        gtbl["SeasWk_RMSE"].to_numpy()
    ]), axis=0)
    gtbl["best_base_RMSE"] = base_rmse
    gtbl["delta_RMSE"] = gtbl["best_base_RMSE"] - gtbl["XGB_RMSE"]
    for (task, H), grp in gtbl.groupby(["task","horizon"]):
        deltas = grp["delta_RMSE"].to_numpy()
        wins   = int(np.sum(deltas > 0))
        summ   = paired_summary(deltas, wins)
        rows.append({"level":"site×fold", "task":task, "horizon":int(H), "pairs":summ["n"],
                     "wins":summ["wins"], "win_rate": wins/max(1,summ["n"]),
                     "mean_delta_RMSE": float(np.mean(deltas)),
                     "p_wilcoxon": summ["p_wilcoxon"], "p_sign": summ["p_sign"], "effect_d": summ["effect_d"]})
    rq1_sig_site = pd.DataFrame(rows).sort_values(["task","horizon"]).reset_index(drop=True)
else:
    rq1_sig_site = pd.DataFrame(columns=["level","task","horizon","pairs","wins","win_rate",
                                         "mean_delta_RMSE","p_wilcoxon","p_sign","effect_d"])

# -------- FDR adjustment across all tests (fold + site×fold) --------
both = pd.concat([rq1_sig_fold, rq1_sig_site], ignore_index=True)
if not both.empty:
    both["p_adj_wilcoxon"] = bh_fdr(both["p_wilcoxon"].fillna(1.0).to_numpy())
    both["p_adj_sign"]     = bh_fdr(both["p_sign"].fillna(1.0).to_numpy())
display(both)


,level,task,horizon,pairs,wins,win_rate,mean_delta_RMSE,p_wilcoxon,p_sign,effect_d,p_adj_wilcoxon,p_adj_sign
0,fold,solar,1,5,5,1.000000,0.039649,3.125000e-02,3.125000e-02,7.663296,3.125000e-02,3.125000e-02
1,fold,solar,24,5,5,1.000000,0.010294,3.125000e-02,3.125000e-02,2.205847,3.125000e-02,3.125000e-02
2,fold,wind,1,5,5,1.000000,0.093504,3.125000e-02,3.125000e-02,12.052679,3.125000e-02,3.125000e-02
3,fold,wind,24,5,5,1.000000,0.280149,3.125000e-02,3.125000e-02,17.449641,3.125000e-02,3.125000e-02
4,site×fold,wind,1,770,768,0.997403,0.091965,5.522189e-128,4.779912e-227,3.528489,1.695881e-127,2.867947e-226
5,site×fold,wind,24,770,767,0.996104,0.275546,5.652936e-128,1.225259e-224,4.234979,1.695881e-127,3.675777e-224


In [ ]:
  # Cell 7 — Power for detecting a 5% MAE improvement (fold-level)
from scipy.stats import norm

def power_calc(deltas, baseline_mean_mae, alpha=0.05, power=0.80, target_rel=0.05):
    d = pd.Series(deltas).dropna()
    if len(d) < 3 or baseline_mean_mae <= 0:
        return dict(n_eff_required=np.nan, n_eff_current=len(d))
    sd = d.std(ddof=1)
    delta = target_rel * baseline_mean_mae
    if sd <= 0 or delta <= 0:
        return dict(n_eff_required=1, n_eff_current=len(d))
    z_alpha = norm.ppf(1 - alpha/2); z_beta = norm.ppf(power)
    n_eff_req = ((z_alpha + z_beta) * sd / delta)**2
    return dict(n_eff_required=float(np.ceil(n_eff_req)), n_eff_current=len(d))

rows = []
for (task, H), grp in cv_table.groupby(["task","horizon"]):
    deltas_mae, base_mae_vals = [], []
    for _, r in grp.iterrows():
        base_mae = np.nanmin([r["Pers_MAE"], r["SeasDay_MAE"], r["SeasWk_MAE"]])
        if pd.notnull(base_mae) and pd.notnull(r["XGB_MAE"]):
            deltas_mae.append(base_mae - r["XGB_MAE"])
            base_mae_vals.append(base_mae)
    if not deltas_mae: continue
    base_mean = float(np.nanmean(base_mae_vals))
    info = power_calc(deltas_mae, base_mean, alpha=0.05, power=0.80, target_rel=0.05)
    rows.append({
        "task": task, "horizon": H,
        "baseline_mean_MAE": base_mean,
        "delta_target(5%)": 0.05 * base_mean,
        "delta_observed_mean": float(np.mean(deltas_mae)),
        "sd_delta": float(np.std(deltas_mae, ddof=1)) if len(deltas_mae)>1 else np.nan,
        **info
    })

power_table = pd.DataFrame(rows).sort_values(["task","horizon"])
display(power_table)


,task,horizon,baseline_mean_MAE,delta_target(5%),delta_observed_mean,sd_delta,n_eff_required,n_eff_current
0,solar,1,0.031047,0.001552,0.014984,0.004276,60.0,5
1,solar,24,0.031048,0.001552,-0.001083,0.005145,87.0,5
2,wind,1,0.298536,0.014927,-0.093205,0.014868,8.0,5
3,wind,24,0.784117,0.039206,-0.047077,0.016299,2.0,5


In [ ]:
# Cell 7b — Power using site×fold pairs AND clustered-by-site means
import numpy as np
import pandas as pd
from scipy.stats import norm

def power_calc_pairs(deltas, baseline_mean_mae, alpha=0.05, power=0.80, target_rel=0.05):
    """
    deltas: array-like of (baseline MAE - model MAE); positive = model better.
    Returns sample size required (paired) to detect a relative target improvement.
    """
    d = pd.Series(deltas).dropna()
    n = len(d)
    if n < 3 or baseline_mean_mae <= 0:
        return dict(n_eff_required=np.nan, n_eff_current=n)
    sd = d.std(ddof=1)
    delta = target_rel * baseline_mean_mae
    if sd <= 0 or delta <= 0:
        return dict(n_eff_required=1.0, n_eff_current=n)
    z_alpha = norm.ppf(1 - alpha/2)
    z_beta  = norm.ppf(power)
    n_req = ((z_alpha + z_beta) * sd / delta) ** 2
    return dict(n_eff_required=float(np.ceil(n_req)), n_eff_current=n)

def _best_base_mae_cols(df):
    """Return a 1-D array of best (min) baseline MAE per row."""
    cols = [c for c in ["Pers_MAE","SeasDay_MAE","SeasWk_MAE"] if c in df.columns]
    arr = np.vstack([df[c].to_numpy() for c in cols])
    return np.nanmin(arr, axis=0)

# -------- Site×fold pairs (more observations) --------
rows_pairs = []
if 'cv_site_table_last' in globals() and not cv_site_table_last.empty:
    for (task, H), grp in cv_site_table_last.groupby(["task","horizon"]):
        base_mae = _best_base_mae_cols(grp)
        deltas   = base_mae - grp["XGB_MAE"].to_numpy()   # + = model better
        base_mean= float(np.nanmean(base_mae))
        info     = power_calc_pairs(deltas, base_mean, alpha=0.05, power=0.80, target_rel=0.05)
        rows_pairs.append({
            "task": task, "horizon": H,
            "pairs(site×fold)": int(len(deltas)),
            "baseline_mean_MAE": base_mean,
            "delta_target(5%)": 0.05 * base_mean,
            "delta_observed_mean": float(np.nanmean(deltas)),
            "sd_delta": float(np.nanstd(deltas, ddof=1)) if len(deltas)>1 else np.nan,
            **info
        })
    power_pairs_table = pd.DataFrame(rows_pairs).sort_values(["task","horizon"])
    print("Power from site×fold pairs")
    display(power_pairs_table)
else:
    print("cv_site_table_last not found — re-run patched Cell 4 and Cell 5 first.")
    power_pairs_table = pd.DataFrame()

# -------- Clustered by site (conservative) --------
rows_cluster = []
if not power_pairs_table.empty:
    for (task, H), grp in cv_site_table_last.groupby(["task","horizon"]):
        base_mae_row = _best_base_mae_cols(grp)
        delta_row    = base_mae_row - grp["XGB_MAE"].to_numpy()

        tmp = grp.copy()
        tmp["base_mae"]  = base_mae_row
        tmp["delta_mae"] = delta_row

        # aggregate to per-site means (treat each site as one paired unit)
        site_delta = tmp.groupby("site")["delta_mae"].mean()
        site_base  = tmp.groupby("site")["base_mae"].mean()
        base_mean  = float(site_base.mean())

        info = power_calc_pairs(site_delta.values, base_mean, alpha=0.05, power=0.80, target_rel=0.05)
        rows_cluster.append({
            "task": task, "horizon": H,
            "sites": int(site_delta.size),
            "baseline_mean_MAE(site-avg)": base_mean,
            "delta_target(5%)": 0.05 * base_mean,
            "delta_observed_mean(site-avg)": float(site_delta.mean()),
            "sd_delta(site-avg)": float(site_delta.std(ddof=1)) if site_delta.size>1 else np.nan,
            **info
        })
    power_cluster_table = pd.DataFrame(rows_cluster).sort_values(["task","horizon"])
    print("Power clustered by site (conservative)")
    display(power_cluster_table)
else:
    power_cluster_table = pd.DataFrame()


Power from site×fold pairs


,task,horizon,pairs(site×fold),baseline_mean_MAE,delta_target(5%),delta_observed_mean,sd_delta,n_eff_required,n_eff_current
0,wind,1,770,0.298219,0.014911,-0.093521,0.029694,32.0,770
1,wind,24,770,0.783296,0.039165,-0.047899,0.071997,27.0,770


Power clustered by site (conservative)


,task,horizon,sites,baseline_mean_MAE(site-avg),delta_target(5%),delta_observed_mean(site-avg),sd_delta(site-avg),n_eff_required,n_eff_current
0,wind,1,154,0.298219,0.014911,-0.093521,0.021793,17.0,154
1,wind,24,154,0.783296,0.039165,-0.047899,0.052749,15.0,154


In [ ]:
# Site×fold power for both tasks using cv_site_table_all (created by the patched CV)
from scipy.stats import norm
import numpy as np

def power_calc_pairs(deltas, baseline_mean_mae, alpha=0.05, power=0.80, target_rel=0.05):
    d = pd.Series(deltas).dropna()
    if len(d) < 3 or baseline_mean_mae <= 0: return dict(n_eff_required=np.nan, n_eff_current=len(d))
    sd = d.std(ddof=1); delta = target_rel * baseline_mean_mae
    z_alpha = norm.ppf(1 - alpha/2); z_beta = norm.ppf(power)
    return dict(n_eff_required=float(np.ceil(((z_alpha + z_beta) * sd / delta)**2)), n_eff_current=len(d))

def site_power(tbl, task):
    g = tbl[tbl['task']==task].copy()
    base = np.nanmin(np.vstack([g['Pers_MAE'], g['SeasDay_MAE'], g['SeasWk_MAE']]), axis=0)
    deltas = base - g['XGB_MAE'].to_numpy()
    base_mean = float(np.nanmean(base))
    info = power_calc_pairs(deltas, base_mean)
    return {'task':task, 'pairs':int((~np.isnan(deltas)).sum()),
            'baseline_mean_MAE':base_mean, 'delta_observed_mean':float(np.nanmean(deltas)),
            **info}

pd.DataFrame([site_power(cv_site_table_all,'solar'),
              site_power(cv_site_table_all,'wind')])


,task,pairs,baseline_mean_MAE,delta_observed_mean,n_eff_required,n_eff_current
0,solar,1540,0.030360,0.006263,470.0,1540
1,wind,1540,0.540758,-0.070710,39.0,1540


In [ ]:
# Cell 7d — Site×fold power (RMSE)
import numpy as np, pandas as pd
from scipy.stats import norm

def power_calc_pairs(deltas, baseline_mean, alpha=0.05, power=0.80, target_rel=0.05):
    d = pd.Series(deltas).dropna()
    n = len(d)
    if n < 3 or baseline_mean <= 0:
        return dict(n_eff_required=np.nan, n_eff_current=n)
    sd = d.std(ddof=1)
    delta = target_rel * baseline_mean
    z_alpha = norm.ppf(1 - alpha/2); z_beta = norm.ppf(power)
    n_req = ((z_alpha + z_beta) * sd / delta) ** 2
    return dict(n_eff_required=float(np.ceil(n_req)), n_eff_current=n)

def rmse_power_from_sitefold(tbl, task):
    g = tbl[tbl["task"] == task].copy()
    # best baseline per row (RMSE)
    base = np.nanmin(np.vstack([g["Pers_RMSE"], g["SeasDay_RMSE"], g["SeasWk_RMSE"]]), axis=0)
    deltas = base - g["XGB_RMSE"].to_numpy()  # + => model better
    base_mean = float(np.nanmean(base))
    info = power_calc_pairs(deltas, base_mean, alpha=0.05, power=0.80, target_rel=0.05)
    return {
        "task": task,
        "pairs": int((~np.isnan(deltas)).sum()),
        "baseline_mean_RMSE": base_mean,
        "delta_observed_mean": float(np.nanmean(deltas)),
        **info
    }

rmse_power_table = pd.DataFrame([
    rmse_power_from_sitefold(cv_site_table_all, "solar"),
    rmse_power_from_sitefold(cv_site_table_all, "wind"),
])
display(rmse_power_table)


In [ ]:
# Cell 8 — FAST RQ2: Site-Only vs Pooled (sparse preproc, site sampling, weights + per-site calibration)

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

# -------- speed knobs for RQ2 --------
RQ2_FAST = True
RQ2_MAX_SITES_PER_FOLD = 8 if RQ2_FAST else None   # cap sites per fold to speed up
RQ2_N_ESTIMATORS = 120 if RQ2_FAST else 300
RQ2_MAX_DEPTH    = 4   if RQ2_FAST else 6
RQ2_LEARNING_RATE= 0.12 if RQ2_FAST else 0.08

def build_preprocessor_sparse(num_cols, cat_cols):
    """One preprocessor per (task,horizon,fold). Outputs sparse matrices."""
    try:
        ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)

    num_trans = Pipeline([
        ("imp", SimpleImputer(strategy="median")),
        ("sc",  StandardScaler(with_mean=False)),
    ])
    cat_trans = Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("ohe", ohe),
    ])
    # force sparse output for speed & memory
    pre = ColumnTransformer(
        [("num", num_trans, num_cols),
         ("cat", cat_trans, cat_cols)],
        remainder="drop", sparse_threshold=1.0
    )
    return pre

def run_rq2_site_vs_pooled(df_task: pd.DataFrame, task: str, horizons=HORIZONS, n_splits=N_SPLITS):
    target = choose_target(task)
    out = []

    base = df_task[[ts_col, site_col, target] + df_task.select_dtypes(include=[np.number]).columns.tolist()].copy()
    base = base.loc[:, ~base.columns.duplicated()].sort_values(ts_col)

    for H in horizons:
        gH, Xc, yc, num_cols, cat_cols = make_supervised(base, target, H)
        if gH.empty:
            continue

        # For aligned baselines on the test block
        def _target_series(frame): return _resolve_target_series(frame, target)

        for fold, (tr_times, te_times) in enumerate(blocked_splits_by_time(gH, n_splits), 1):
            tr_pool = gH[gH[ts_col].isin(tr_times)].copy()
            te_all  = gH[gH[ts_col].isin(te_times)].copy()
            if len(tr_pool) < 200 or len(te_all) < 60:
                continue

            # ----- 1) Fit ONE preprocessor on pooled train, transform train/test once
            pre = build_preprocessor_sparse(num_cols, cat_cols)
            Xt_tr_pool = pre.fit_transform(tr_pool[Xc])   # scipy.sparse
            Xt_te_all  = pre.transform(te_all[Xc])

            y_tr_pool = tr_pool[yc].to_numpy()

            # ----- 2) Train ONE pooled model on all rows (with per-site weights)
            pooled = XGBRegressor(
                n_estimators=RQ2_N_ESTIMATORS,
                learning_rate=RQ2_LEARNING_RATE,
                max_depth=RQ2_MAX_DEPTH,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_lambda=1.0,
                tree_method="hist",
                n_jobs=-1,
                random_state=RANDOM_STATE,
                verbosity=0,
            )
            counts = tr_pool[site_col].value_counts()
            w_pool = tr_pool[site_col].map(lambda s: 1.0 / max(1, counts.get(s, 1))).to_numpy()
            w_pool = w_pool / w_pool.mean()

            pooled.fit(Xt_tr_pool, y_tr_pool, sample_weight=w_pool)

            # ----- 3) Select sites for this fold (optionally cap to fastest subset)
            sites = te_all[site_col].unique().tolist()
            if RQ2_MAX_SITES_PER_FOLD:
                # choose sites with the largest site-only training size first
                sizes = tr_pool[site_col].value_counts()
                sites.sort(key=lambda s: int(sizes.get(s, 0)), reverse=True)
                sites = sites[:RQ2_MAX_SITES_PER_FOLD]

            # Precompute aligned baselines on test block (once)
            t_te = _target_series(te_all)
            pers_all    = t_te.groupby(te_all[site_col]).shift(0).to_numpy()
            seas_dy_all = t_te.groupby(te_all[site_col]).shift(24 - H).to_numpy()
            seas_wk_all = t_te.groupby(te_all[site_col]).shift(168 - H).to_numpy()

            # ----- 4) Loop sites: train a small site-only model & evaluate pooled (raw vs calibrated)
            for sid in sites:
                mask_tr_site = (tr_pool[site_col].values == sid)
                mask_te_site = (te_all[site_col].values == sid)

                # skip tiny splits
                if mask_tr_site.sum() < 150 or mask_te_site.sum() < 60:
                    continue

                Xt_tr_site = Xt_tr_pool[mask_tr_site]
                y_tr_site  = tr_pool.loc[mask_tr_site, yc].to_numpy()

                Xt_te_site = Xt_te_all[mask_te_site]
                ytrue      = te_all.loc[mask_te_site, yc].to_numpy()

                # small site-only model
                site_only = XGBRegressor(
                    n_estimators=RQ2_N_ESTIMATORS,
                    learning_rate=RQ2_LEARNING_RATE,
                    max_depth=RQ2_MAX_DEPTH,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_lambda=1.0,
                    tree_method="hist",
                    n_jobs=-1,
                    random_state=RANDOM_STATE,
                    verbosity=0,
                )
                site_only.fit(Xt_tr_site, y_tr_site)

                # predictions (site-only and pooled)
                yhat_site = site_only.predict(Xt_te_site)

                # pooled raw predictions on this site's rows
                yhat_pool_raw = pooled.predict(Xt_te_site)

                # ---- NEW: per-site linear calibration of pooled predictions
                # Fit a tiny ridge on pooled predictions (1-D) using this site's TRAIN data
                pooled_tr_site = pooled.predict(Xt_tr_site)
                try:
                    cal = Ridge(alpha=1.0, random_state=RANDOM_STATE).fit(
                        pooled_tr_site.reshape(-1, 1), y_tr_site
                    )
                    yhat_pool_cal = cal.predict(yhat_pool_raw.reshape(-1, 1))
                    # pick the better of raw vs calibrated (by RMSE)
                    m_pool_raw = eval_metrics(ytrue, yhat_pool_raw)
                    m_pool_cal = eval_metrics(ytrue, yhat_pool_cal)
                    if m_pool_cal["RMSE"] <= m_pool_raw["RMSE"]:
                        yhat_pool = yhat_pool_cal
                        m_pool = m_pool_cal
                        pool_tag = "PooledCal"
                    else:
                        yhat_pool = yhat_pool_raw
                        m_pool = m_pool_raw
                        pool_tag = "Pooled"
                except Exception:
                    # fallback: no calibration if something odd happens
                    yhat_pool = yhat_pool_raw
                    m_pool = eval_metrics(ytrue, yhat_pool_raw)
                    pool_tag = "Pooled"

                # metrics
                m_site = eval_metrics(ytrue, yhat_site)

                out.append({
                    "task": task, "horizon": H, "fold": fold, "site": sid,
                    "SiteOnly_RMSE": m_site["RMSE"], "Pooled_RMSE": m_pool["RMSE"],
                    "SiteOnly_MAE":  m_site["MAE"],  "Pooled_MAE":  m_pool["MAE"],
                    "pooled_variant": pool_tag,
                    "n_train_site": int(mask_tr_site.sum()),
                    "n_train_pooled": int(Xt_tr_pool.shape[0]),
                    "n_test": int(mask_te_site.sum())
                })

            # (optional) minimal progress print
            print(f"[RQ2] {task} H={H} fold={fold}: processed {len(sites)} site(s)")

    return pd.DataFrame(out)

# ---- run (same as before) ----
rq2_tbls = []
if solar_col: rq2_tbls.append(run_rq2_site_vs_pooled(df, "solar"))
if wind_col:  rq2_tbls.append(run_rq2_site_vs_pooled(df, "wind"))
rq2_table = pd.concat(rq2_tbls, ignore_index=True) if rq2_tbls else pd.DataFrame()
display(rq2_table.head(10))


[RQ2] solar H=1 fold=1: processed 8 site(s)
[RQ2] solar H=1 fold=2: processed 8 site(s)
[RQ2] solar H=1 fold=3: processed 8 site(s)
[RQ2] solar H=1 fold=4: processed 8 site(s)
[RQ2] solar H=1 fold=5: processed 8 site(s)
[RQ2] solar H=24 fold=1: processed 8 site(s)
[RQ2] solar H=24 fold=2: processed 8 site(s)
[RQ2] solar H=24 fold=3: processed 8 site(s)
[RQ2] solar H=24 fold=4: processed 8 site(s)
[RQ2] solar H=24 fold=5: processed 8 site(s)
[RQ2] wind H=1 fold=1: processed 8 site(s)
[RQ2] wind H=1 fold=2: processed 8 site(s)
[RQ2] wind H=1 fold=3: processed 8 site(s)
[RQ2] wind H=1 fold=4: processed 8 site(s)
[RQ2] wind H=1 fold=5: processed 8 site(s)
[RQ2] wind H=24 fold=1: processed 8 site(s)
[RQ2] wind H=24 fold=2: processed 8 site(s)
[RQ2] wind H=24 fold=3: processed 8 site(s)
[RQ2] wind H=24 fold=4: processed 8 site(s)
[RQ2] wind H=24 fold=5: processed 8 site(s)


,task,horizon,fold,site,SiteOnly_RMSE,Pooled_RMSE,SiteOnly_MAE,Pooled_MAE,pooled_variant,n_train_site,n_train_pooled,n_test
0,solar,1,1,MX_PuertoPenasco_Son,0.026063,0.035648,0.013083,0.021677,Pooled,2897,446138,2920
1,solar,1,1,ES_Pamplona,0.042398,0.039481,0.024772,0.025163,Pooled,2897,446138,2920
2,solar,1,1,TH_NakhonRatchasima,0.055564,0.050927,0.032175,0.027537,Pooled,2897,446138,2920
3,solar,1,1,IN_Kadapa_AP,0.044386,0.041616,0.026017,0.024185,Pooled,2897,446138,2920
4,solar,1,1,KR_Ulsan,0.044362,0.046630,0.025590,0.026627,PooledCal,2897,446138,2920
5,solar,1,1,IE_Galway,0.045396,0.042922,0.027314,0.026604,Pooled,2897,446138,2920
6,solar,1,1,US_OK_Woodward,0.033848,0.040435,0.018096,0.024399,PooledCal,2897,446138,2920
7,solar,1,1,IT_MontaltoDiCastro,0.037667,0.038456,0.022294,0.025018,PooledCal,2897,446138,2920
8,solar,1,2,FR_Narbonne,0.036982,0.038517,0.018211,0.022665,Pooled,5817,895818,2920
9,solar,1,2,JP_Minamisoma_Fukushima,0.047263,0.039523,0.027809,0.022783,PooledCal,5817,895818,2920


In [ ]:
# Paired tests for RQ2 using rq2_table
from scipy.stats import wilcoxon, binomtest
import numpy as np, pandas as pd

def rq2_significance(tbl):
    rows=[]
    for (task,H), g in tbl.groupby(['task','horizon']):
        d = (g['SiteOnly_RMSE'] - g['Pooled_RMSE']).to_numpy()  # + means pooled better
        n = len(d); wins = int((d > 0).sum())
        p_w = wilcoxon(d, alternative='greater').pvalue if n >= 3 else np.nan
        p_s = binomtest(wins, n, 0.5, alternative='greater').pvalue
        rows.append({
            'task':task, 'horizon':int(H), 'pairs':n,
            'win_rate': wins/max(1,n), 'mean_gain_RMSE': float(np.nanmean(d)),
            'wilcoxon_p': p_w, 'sign_p': p_s
        })
    return pd.DataFrame(rows).sort_values(['task','horizon'])
rq2_sig = rq2_significance(rq2_table)
display(rq2_sig)


,task,horizon,pairs,win_rate,mean_gain_RMSE,wilcoxon_p,sign_p
0,solar,1,40,0.175,-0.002193,0.999876,0.999996
1,solar,24,40,0.725,0.004837,0.000124,0.003213
2,wind,1,40,0.750,0.034408,0.000190,0.001111
3,wind,24,40,0.850,0.070579,0.000001,0.000004


In [ ]:
# Cell 8c — RQ2 stats: effect sizes, FDR, and reverse-direction test
from scipy.stats import wilcoxon, binomtest
import numpy as np, pandas as pd

def bh(pvals):
    p = np.array([1.0 if pd.isna(x) else x for x in pvals], float)
    n = len(p); order = np.argsort(p); ranked = p[order]
    adj = np.empty_like(ranked); m = 1.0
    for i in range(n-1, -1, -1):
        m = min(m, ranked[i] * n / (i+1))
        adj[i] = m
    out = np.empty_like(adj); out[order] = adj
    return out

def rq2_stats(tbl):
    rows=[]
    for (task,H), g in tbl.groupby(['task','horizon']):
        d = (g['SiteOnly_RMSE'] - g['Pooled_RMSE']).to_numpy()  # + => pooled better
        n = len(d); wins = int((d > 0).sum())
        p_w_greater = wilcoxon(d, alternative='greater').pvalue if n>=3 else np.nan
        p_w_less    = wilcoxon(d, alternative='less').pvalue    if n>=3 else np.nan  # H1: pooled worse
        p_sign_g    = binomtest(wins, n, 0.5, alternative='greater').pvalue
        sd = np.std(d, ddof=1) if n>1 else np.nan
        effect_d = float(np.mean(d)/sd) if (sd and sd>0) else np.nan
        rows.append(dict(task=task, horizon=int(H), pairs=n, win_rate=wins/max(1,n),
                         mean_gain_RMSE=float(np.mean(d)),
                         wilcoxon_p_greater=p_w_greater, wilcoxon_p_less=p_w_less,
                         sign_p_greater=p_sign_g, effect_d=effect_d))
    out = pd.DataFrame(rows).sort_values(['task','horizon'])
    out['p_adj_wilcoxon_greater'] = bh(out['wilcoxon_p_greater'])
    out['p_adj_sign_greater']     = bh(out['sign_p_greater'])
    display(out)
    return out

rq2_sig_detailed = rq2_stats(rq2_table)


,task,horizon,pairs,win_rate,mean_gain_RMSE,wilcoxon_p_greater,wilcoxon_p_less,sign_p_greater,effect_d,p_adj_wilcoxon_greater,p_adj_sign_greater
0,solar,1,40,0.175,-0.002193,0.999876,0.000132,0.999996,-0.497974,0.999876,0.999996
1,solar,24,40,0.725,0.004837,0.000124,0.999883,0.003213,0.577636,0.000248,0.004284
2,wind,1,40,0.750,0.034408,0.000190,0.999821,0.001111,0.600001,0.000253,0.002221
3,wind,24,40,0.850,0.070579,0.000001,0.999999,0.000004,0.799877,0.000004,0.000017


In [ ]:
# Cell 10b — Narrative lines including significance and effects

def narrative_from_sig(df):
    lines = []
    for _, r in df.sort_values(["level","task","horizon"]).iterrows():
        sig = "significant" if (pd.notnull(r["p_adj_wilcoxon"]) and r["p_adj_wilcoxon"] < 0.05) else "not significant"
        lines.append(
            f"{r['level']} — {r['task']} @H={int(r['horizon'])}: "
            f"pairs={int(r['pairs'])}, win-rate={r['win_rate']:.2f}, "
            f"ΔRMSE={r['mean_delta_RMSE']:.4f}, "
            f"Wilcoxon p={r['p_wilcoxon']:.3g} (FDR adj {r['p_adj_wilcoxon']:.3g}) → {sig}, "
            f"sign test p={r['p_sign']:.3g}, effect d={r['effect_d'] if pd.notnull(r['effect_d']) else 'NA'}."
        )
    return "\n".join(lines)

if 'both' in globals() and not both.empty:
    print(narrative_from_sig(both))
else:
    print("Run Cell 6b first.")


fold — solar @H=1: pairs=5, win-rate=1.00, ΔRMSE=0.0396, Wilcoxon p=0.0312 (FDR adj 0.0312) → significant, sign test p=0.0312, effect d=7.663295538748312.
fold — solar @H=24: pairs=5, win-rate=1.00, ΔRMSE=0.0103, Wilcoxon p=0.0312 (FDR adj 0.0312) → significant, sign test p=0.0312, effect d=2.2058465987529288.
fold — wind @H=1: pairs=5, win-rate=1.00, ΔRMSE=0.0935, Wilcoxon p=0.0312 (FDR adj 0.0312) → significant, sign test p=0.0312, effect d=12.052679332786553.
fold — wind @H=24: pairs=5, win-rate=1.00, ΔRMSE=0.2801, Wilcoxon p=0.0312 (FDR adj 0.0312) → significant, sign test p=0.0312, effect d=17.449640847479287.
site×fold — wind @H=1: pairs=770, win-rate=1.00, ΔRMSE=0.0920, Wilcoxon p=5.52e-128 (FDR adj 1.7e-127) → significant, sign test p=4.78e-227, effect d=3.528489276198357.
site×fold — wind @H=24: pairs=770, win-rate=1.00, ΔRMSE=0.2755, Wilcoxon p=5.65e-128 (FDR adj 1.7e-127) → significant, sign test p=1.23e-224, effect d=4.234978579754188.
